# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [44]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [45]:
weather_df = pd.read_csv("../output_data/weather_data.csv")
weather_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Wind Speed,Cloudiness,Country,Date
0,0,Pandan Niog,6.2282,120.5631,83.14,70,4.07,17,PH,1619741475
1,1,Chui,-33.6971,-53.4616,56.86,77,3.09,0,UY,1619741475
2,2,Punta Arenas,-53.1500,-70.9167,41.00,93,8.05,90,CL,1619741476
3,3,Puerto Ayora,-0.7393,-90.3518,76.62,80,9.04,46,EC,1619741476
4,4,Ancud,-41.8697,-73.8203,57.99,88,16.11,75,CL,1619741476


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [72]:
gmaps.configure(api_key=g_key)

locations = weather_df[["Lat", "Lng"]]

humidity = weather_df["Humidity"].astype(float)


In [73]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=10, point_radius=1)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [64]:
narrow_cities = weather_df.loc[(weather_df['Max Temp'] > 70) & (weather_df['Max Temp'] < 80) & (weather_df['Wind Speed'] < 10) & (weather_df['Cloudiness'] == 0)].dropna() 

narrow_cities = narrow_cities.drop(columns = ['Unnamed: 0'])
narrow_cities

,City,Lat,Lng,Max Temp,Humidity,Wind Speed,Cloudiness,Country,Date
50,Shanghai,31.2222,121.4581,71.01,60,6.71,0,CN,1619741464
115,Inhambane,-23.8650,35.3833,73.06,79,7.87,0,MZ,1619741521
155,Sakakah,29.9697,40.2064,71.60,16,4.61,0,SA,1619741538
223,Conceição do Araguaia,-8.2578,-49.2647,75.20,94,4.61,0,BR,1619741566
271,Cayenne,4.9333,-52.3333,78.01,94,4.09,0,GF,1619741585
285,Teacapan,22.5500,-105.7500,78.12,58,9.71,0,MX,1619741591
354,Manicoré,-5.8092,-61.3003,75.20,100,2.30,0,BR,1619741619
421,Beira,-19.8436,34.8389,75.20,88,5.66,0,MZ,1619741648
451,Beloha,-25.1667,45.0500,72.19,67,6.24,0,MG,1619741659
452,Khorramshahr,30.4397,48.1664,77.00,41,4.61,0,IR,1619741659


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [94]:
hotel_df = narrow_cities.loc[:,["City", "Country", "Lat", "Lng"]]

hotel_df["Hotel Name"] = " "
hotel_df

,City,Country,Lat,Lng,Hotel Name
50,Shanghai,CN,31.2222,121.4581,
115,Inhambane,MZ,-23.8650,35.3833,
155,Sakakah,SA,29.9697,40.2064,
223,Conceição do Araguaia,BR,-8.2578,-49.2647,
271,Cayenne,GF,4.9333,-52.3333,
285,Teacapan,MX,22.5500,-105.7500,
354,Manicoré,BR,-5.8092,-61.3003,
421,Beira,MZ,-19.8436,34.8389,
451,Beloha,MG,-25.1667,45.0500,
452,Khorramshahr,IR,30.4397,48.1664,


In [95]:
print("---------------")
print("Begining Search")
print("---------------")


for index, row in hotel_df.iterrows():
    Lat = row["Lat"]
    Lng = row["Lng"]
    city = row["City"]
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" 
    
    params = {"location" : f"{Lat},{Lng}", 
        "radius" : 5000,
        "types" : "hotel",
        "key" : g_key }
    params["location"] = f"{Lat}, {Lng}"
    
    print(f"Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        print(f"Hotel closest to {city} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("missing results")
        
        print("---------------")
        print("End Search")
        print("---------------")

---------------
Begining Search
---------------
Results for Index 50: Shanghai.
Hotel closest to Shanghai is Shanghai.
Results for Index 115: Inhambane.
Hotel closest to Inhambane is Maxixe.
Results for Index 155: Sakakah.
Hotel closest to Sakakah is Sakaka.
Results for Index 223: Conceição do Araguaia.
Hotel closest to Conceição do Araguaia is Conceição do Araguaia.
Results for Index 271: Cayenne.
Hotel closest to Cayenne is Cayenne.
Results for Index 285: Teacapan.
Hotel closest to Teacapan is Teacapan.
Results for Index 354: Manicoré.
Hotel closest to Manicoré is Manicoré.
Results for Index 421: Beira.
Hotel closest to Beira is Beira.
Results for Index 451: Beloha.
Hotel closest to Beloha is Beloha.
Results for Index 452: Khorramshahr.
Hotel closest to Khorramshahr is Khorramshahr.
Results for Index 458: Bhuj.
Hotel closest to Bhuj is Bhuj.
Results for Index 516: Camalú.
Hotel closest to Camalú is Camalú.


In [96]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
50,Shanghai,CN,31.2222,121.4581,Shanghai
115,Inhambane,MZ,-23.8650,35.3833,Maxixe
155,Sakakah,SA,29.9697,40.2064,Sakaka
223,Conceição do Araguaia,BR,-8.2578,-49.2647,Conceição do Araguaia
271,Cayenne,GF,4.9333,-52.3333,Cayenne
285,Teacapan,MX,22.5500,-105.7500,Teacapan
354,Manicoré,BR,-5.8092,-61.3003,Manicoré
421,Beira,MZ,-19.8436,34.8389,Beira
451,Beloha,MG,-25.1667,45.0500,Beloha
452,Khorramshahr,IR,30.4397,48.1664,Khorramshahr


In [97]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [99]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))